# Цель лабораторной работы
Целью данной работы является реализация алгоритма расчета степени близости объектов, описываемых набором численно выраженных свойств с использованием технологии CUDA.

# Постановка задачи
Мы имеем набор объектов, которые могут быть описаны наборами свойств, выраженных численно. В этой задаче необходимо рассчитать степень сходства объектов. В качестве входных параметров имеем матрицу:

![](images/CropperScreenshot_2019-06-21_21:35:23.png)

Надо вычислить расстояние между каждой возможной парой строк матрицы.

![](images/CropperScreenshot_2019-06-21_21:35:41.png)

В работе предполагается реализация данного алгоритма с использованием программно-аппаратной архитектуры параллельных вычислений CUDA с глобальной и общей памятью. Необходимо определить конфигурацию ядра и выполнить тесты ускорения с различными примерами входных параметров.

# Краткая Теория
Технология CUDA вводит ряд дополнительных расширений для языка C, которые необходимы для написания кода для GPU:
1. Функция указывает, как и откуда будут выполняться функции;
2. Описатели переменных, которые служат для указания типа памяти, используемой графическим процессором;
3. Квалификаторы запуска ядра GPU;
4. Встроенные переменные для идентификации потоков, блоков и других параметров при выполнении кода в ядре GPU.
5. Дополнительные типы переменных.


Спецификаторы функций определяют, как и откуда будут вызываться функции. В CUDA есть 3 таких спецификатора:
- __host__ 
- __global__ 
- __device__ 


Квалификаторы запуска ядра используются для описания количества блоков, потоков и памяти, которые необходимо выделить при вычислении на GPU. 
API хоста CUDA - это связь между процессором и графическим процессором.


# Алгоритм реализации

In [1]:
%cat mm.cu

#include <cuda_runtime.h>

#include <iostream>
#include <memory>
#include <string>

#include <cuda.h>
#include <stdio.h>




#ifndef BLOCK_SIZE
# define BLOCK_SIZE 16
#endif

#ifndef _M
# define _M 10000
#endif

#ifndef _N
# define _N 10000
#endif

#if !defined(CUDA) && !defined(CPU) && !defined(CHECK)
# define CUDA
#endif

#define gpuErrchk(ans) { gpuAssert((ans), __FILE__, __LINE__); }
inline void gpuAssert(cudaError_t code, const char *file, int line, bool abort=true)
{
   if (code != cudaSuccess) 
   {
      fprintf(stderr,"gpuAssert: %s %s %d\n", cudaGetErrorString(code), file, line);
      if (abort) exit(code);
   }
}

__global__ void mx_dist(float *m_in, float *m_out, int m, int n) 
{
    int i = blockIdx.y * blockDim.y + threadIdx.y; 
    int j = blockIdx.x * blockDim.x + threadIdx.x;
	float s = 0, sum = 0;

    if( i < m && j < m) {

    	for(int k = 0; k < n; ++k) {
    		s = m_in[i*m + k] - m_in[j*m + k];
    		sum += s*s;
    	}

    	// printf("--> %d %d %f %f\n", j, i, m

# Результаты

In [2]:
import subprocess
import os

def compile(*defs, **defskw):
    args = [f"-D{k}" for k in defs] + [f"-D{k}={v}" for k, v in defskw.items()]
    _cmd = 'nvcc mm.cu -o mm'.split() + args
    # print(' '.join(_cmd))
    cmd = subprocess.run(_cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  
    if(cmd.stdout): print('cmd.stdout', cmd.stdout)
    if(cmd.stderr): print('cmd.stderr', cmd.stderr)
    
def run(env=None):
    cmd = subprocess.run('./mm', stdout=subprocess.PIPE, stderr=subprocess.PIPE, env=env)
    return cmd.stdout.decode('utf8')    




Давайте посмотрим, как производительность на CPU сравнивается с производительностью на GPU

In [3]:
compile('CPU', _N=10_00, _M=50_0)
print("Execution time on CPU", end="\n\t")
%timeit run()
print()

compile('CUDA', _N=10_00, _M=50_0)
print("Execution time on GPU", end="\n\t")
%timeit run()
print()

Execution time on CPU
	1.2 s ± 371 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

Execution time on GPU
	19.5 ms ± 394 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)



Мы видим явный выигрыш от выполнения GPU.

Давайте посмотрим, как размер блока влияет на производительность.

In [4]:

for bs in [8, 16, 32, 64, 128, 256, 512, 1024, 2048]:
    compile('CUDA', _N=10_00, _M=50_0, BLOCK_SIZE=bs)
    print(f"Execution time with block size {bs}", end="\n\t")
    %timeit run()
    print()



Execution time with block size 8
	29.7 ms ± 698 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)

Execution time with block size 16
	21.6 ms ± 2.95 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)

Execution time with block size 32
	28.4 ms ± 1.82 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

Execution time with block size 64
	21.2 ms ± 3.38 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

Execution time with block size 128
	21.5 ms ± 2.84 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)

Execution time with block size 256
	21.8 ms ± 2.64 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

Execution time with block size 512
	24.2 ms ± 1.79 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

Execution time with block size 1024
	22.2 ms ± 940 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)

Execution time with block size 2048
	22.6 ms ± 3.02 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)



# Вывод

В данной работе реализован алгоритм расчета степени близости объектов, описываемых набором численно выраженных свойств с использованием технологии CUDA. Тесты проводились для ядра с глобальной и общей памятью.
В результате этой работы был получен навык работы с технологией CUDA написания кода для GPU.